In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/time-series/2015_Zika.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import os

# ========================
# 1. Đọc & xử lý dữ liệu
# ========================
file_path = "/kaggle/input/time-series/2015_Zika.csv"

try:
    df = pd.read_csv(file_path, on_bad_lines='skip')  # pandas >=1.3.0
except Exception as e:
    print(f"Lỗi khi đọc file CSV: {e}")

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date']).sort_values(by='date').reset_index(drop=True)
df = df.drop_duplicates(subset=['username', 'text'])
df['OnlyDate'] = df['date'].dt.date

df_daily = df.groupby('OnlyDate').agg({
    'retweets': 'sum',
    'favorites': 'sum',
    'replies': 'sum',
    'text': lambda x: x.astype(str).str.len().mean(),
    'hashtags': lambda x: x.astype(str).str.len().mean()
}).rename(columns={
    'retweets': 'retweets_sum',
    'favorites': 'favorites_sum',
    'replies': 'replies_sum',
    'text': 'text_length',
    'hashtags': 'hashtag_length'
})

article_counts = df.groupby('OnlyDate').size()
df_daily['num_articles'] = article_counts

full_index = pd.date_range(start=df_daily.index.min(), end=df_daily.index.max(), freq='D')
df_daily = df_daily.reindex(full_index).fillna(0)
df_daily.index.name = 'date'

# ========================
# 2. Chuẩn hóa & tạo dữ liệu
# ========================
def create_multivariate_dataset(data, window_size=20, target_column=0):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size].T)
        y.append(data[i + window_size, target_column])
    return np.array(X), np.array(y)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_daily.values)

window_size = 20
target_column = df_daily.columns.get_loc('num_articles')
X_np, y_np = create_multivariate_dataset(data_scaled, window_size, target_column)

X_tensor = torch.tensor(X_np, dtype=torch.float32)
y_tensor = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# ========================
# 3. Mô hình TCN
# ========================
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size]

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, padding, dropout=0.3):
        super().__init__()
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.utils.weight_norm(nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.dropout1(self.relu1(self.chomp1(self.conv1(x))))
        out = self.dropout2(self.relu2(self.chomp2(self.conv2(out))))
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.3):
        super().__init__()
        layers = []
        for i in range(len(num_channels)):
            dilation = 2 ** i
            in_ch = num_inputs if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            padding = (kernel_size - 1) * dilation
            layers.append(TemporalBlock(in_ch, out_ch, kernel_size, dilation, padding, dropout))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCNModel(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.tcn = TemporalConvNet(input_channels, [32, 64, 32])
        self.linear = nn.Linear(32, 1)

    def forward(self, x):
        y = self.tcn(x)
        return self.linear(y[:, :, -1])

# ========================
# 4. Huấn luyện mô hình
# ========================
model = TCNModel(input_channels=X_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_loss = float('inf')
patience = 10
counter = 0
epochs = 300
loss_history = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()

    val_loss = loss.item()
    loss_history.append(val_loss)

    print(f"Epoch {epoch+1}: Loss = {val_loss:.6f}")
    if val_loss < best_loss - 1e-5:
        best_loss = val_loss
        best_model = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"⛔ Early stopping at epoch {epoch+1}")
            break

model.load_state_dict(best_model)

# ========================
# 5. Đánh giá mô hình
# ========================
model.eval()
with torch.no_grad():
    predictions = model(X_tensor).numpy()

zeros_pad = np.zeros((len(predictions), data_scaled.shape[1]))
zeros_pad[:, target_column] = predictions.ravel()
y_pred = scaler.inverse_transform(zeros_pad)[:, target_column]

zeros_pad[:, target_column] = y_np.ravel()
y_true = scaler.inverse_transform(zeros_pad)[:, target_column]

print("\n📊 Evaluation Metrics:")
print(f"R2 Score:  {r2_score(y_true, y_pred):.4f}")
print(f"MSE:       {mean_squared_error(y_true, y_pred):.4f}")
print(f"MAE:       {mean_absolute_error(y_true, y_pred):.4f}")

# ========================
# 5.5. Phân loại kết quả dự đoán thành 3 mức
# ========================
low_thresh = np.percentile(y_pred, 33)
high_thresh = np.percentile(y_pred, 66)

def classify_level(value):
    if value < low_thresh:
        return "Thấp"
    elif value < high_thresh:
        return "Trung bình"
    else:
        return "Cao"

pred_levels = np.array([classify_level(val) for val in y_pred])
dates_pred = df_daily.index[window_size:]

print("\n🧾 Kết quả phân loại dự đoán (5 dòng đầu):")
for i in range(5):
    print(f"Ngày {dates_pred[i]}: Dự đoán = {y_pred[i]:.2f} → Mức = {pred_levels[i]}")

# ========================
# 6. Biểu đồ kết quả
# ========================
plt.figure(figsize=(12, 5))
plt.plot(y_true, label='Thực tế', linewidth=2)
plt.plot(y_pred, label='Dự đoán', linestyle='--')
plt.title("Dự đoán số bài báo mỗi ngày bằng TCN")
plt.xlabel("Ngày")
plt.ylabel("Số bài báo")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# ========================
# 7. Biểu đồ Loss theo Epoch
# ========================
plt.figure(figsize=(10, 4))
plt.plot(loss_history, color='blue', linewidth=2)
plt.title("Biểu đồ Loss theo Epoch", fontsize=14)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss (MSE)", fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#Lưu mô hình đã huấn luyện (Zika)
# ========================
model_path = "tcn_zika_model.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'input_channels': X_tensor.shape[1]
}, model_path)

print(f"\n✅ Mô hình TCN (Zika) đã được lưu tại: {model_path}")